In [ ]:
# https://drive.google.com/file/d/1sb6AS8JH6jxScJP9ZkIB4QTW5WIDHQTW/view?usp=sharing
!conda install -y gdown
!gdown --id "1sb6AS8JH6jxScJP9ZkIB4QTW5WIDHQTW"
!unzip -q SWIR.zip
!rm SWIR.zip

In [1]:
import os;
import torch;
import torch.fft;
import torchvision;
import numpy as np;
from tqdm import tqdm;
from PIL import Image;
from typing import Tuple;
from random import randint;
from torch import nn, optim;
from torch.utils import data;
from einops import rearrange;
import torch.nn.functional as F;
from torch.backends import cudnn;
from torch.utils.data import Dataset;
from torch.nn.init import trunc_normal_;
from torch.nn.parameter import Parameter;
import torchvision.transforms.functional as FT;

In [2]:
cudnn.benchmark = True;

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu");

batch_size = 64;
learning_rate = 1e-3;
epochs = 8000;

In [3]:
class WindowSelfAttention(nn.Module):

    def __init__(self, dim: int, num_heads: int = 8, window_size: int = 8) -> None:
        super().__init__();
        self.num_heads = num_heads;
        self.window_size = window_size;
        head_dim = dim // num_heads;
        self.scale = head_dim ** -0.5;
        self.query = nn.Conv1d(dim, dim, kernel_size = 1, padding = 0, bias = False);
        self.key = nn.Conv1d(dim, dim, kernel_size = 1, padding = 0, bias = False);
        self.value = nn.Conv1d(dim, dim, kernel_size = 1, padding = 0, bias = False);
        self.beta = nn.Parameter(torch.zeros(num_heads, window_size ** 2, window_size ** 2));
        self.proj_out = nn.Conv1d(dim, dim, kernel_size = 1, padding = 0, bias = True);
        trunc_normal_(self.beta, std = 0.02);

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, c, h, w = x.shape;
        # Pad
        pad_r = (self.window_size - w % self.window_size) % self.window_size;
        pad_b = (self.window_size - h % self.window_size) % self.window_size;
        x = F.pad(x, (0, pad_r, 0, pad_b));
        # Window partition
        x = rearrange(x, 'b c (h s1) (w s2) -> (b h w) c (s1 s2)', s1 = self.window_size, s2 = self.window_size);
        # Project
        q = self.query(x);
        k = self.key(x);
        v = self.value(x);
        # Attention
        q, k, v = map(lambda t: rearrange(t, 'b (h d) n -> b h d n', h = self.num_heads), [q, k, v]);
        attn = torch.einsum('b h d n, b h d m -> b h n m', q, k) * self.scale + self.beta;
        attn = attn.softmax(dim = -1);
        x = torch.einsum('b h n m, b h d m -> b h d n', attn, v);
        x = rearrange(x, 'b i c j -> b (i c) j');
        x = self.proj_out(x);
        # Reverse window partition
        x = rearrange(x, 'B c (s1 s2) -> B c s1 s2', s1 = self.window_size, s2 = self.window_size);
        b = int(x.shape[0] / (h * w / self.window_size / self.window_size));
        x = rearrange(x, '(b h w) c s1 s2 -> b c (h s1) (w s2)', b = b, h = (h + pad_b) // self.window_size, w = (w + pad_r) // self.window_size);
        if((pad_r > 0) or (pad_b > 0)):
            x = x[:, :, :h, :w].contiguous();
        return x;

class PCFN(nn.Module):

    def __init__(self, dim: int, growth_rate: float = 2.0, p_rate: float = 0.25) -> None:
        super().__init__();
        hidden_dim = int(dim * growth_rate);
        p_dim = int(hidden_dim * p_rate);
        self.conv_0 = nn.Conv2d(dim, hidden_dim, kernel_size = 1, padding = 0);
        self.conv_1 = nn.Conv2d(p_dim, p_dim, kernel_size = 3, padding = 1, padding_mode = "reflect");
        self.act = nn.GELU();
        self.conv_2 = nn.Conv2d(hidden_dim, dim, kernel_size = 1, padding = 0);
        self.p_dim = p_dim;
        self.hidden_dim = hidden_dim;

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if(self.training):
            x = self.act(self.conv_0(x));
            x1, x2 = torch.split(x, [self.p_dim, self.hidden_dim - self.p_dim], dim = 1);
            x1 = self.act(self.conv_1(x1));
            x = self.conv_2(torch.cat([x1, x2], dim = 1));
        else:
            x = self.act(self.conv_0(x));
            x[:, :self.p_dim, :, :] = self.act(self.conv_1(x[:, :self.p_dim, :, :]));
            x = self.conv_2(x);
        return x;

class Transformer(nn.Module):

    def __init__(self,
                 dim: int,
                 num_heads: int = 4,
                 window_size: int = 8,
                 mlp_ratio: int = 4) -> None:
        super().__init__();
        self.attn = WindowSelfAttention(dim, num_heads, window_size);
        self.pcfn = PCFN(dim, mlp_ratio);

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.attn(F.normalize(x)) + x;
        x = self.pcfn(F.normalize(x)) + x;
        return x;

class SRViT(nn.Module):

    def __init__(self, n_feats: int = 40, n_heads: int = 8, ratio: int = 2, blocks = 5, upscaling_factor: int = 4) -> None:
        super(SRViT, self).__init__();
        self.head = nn.Conv2d(3, n_feats, 3, 1, 1);
        self.body = nn.Sequential(*[Transformer(n_feats, n_heads, 8, ratio) for i in range(blocks)]);
        self.upsampling = nn.Sequential(
            nn.Conv2d(n_feats, 3 * upscaling_factor ** 2, kernel_size = 3, padding = 1, padding_mode = "reflect"),
            nn.PixelShuffle(upscaling_factor)
        );
        
    def forward(self, x):
        res = F.interpolate(x, size = (x.size(2) * 4, x.size(3) * 4));
        x = self.head(x);
        x = self.upsampling(self.body(x) + x);
        return x + res;

In [4]:
def transform(input: torch.Tensor, target: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    if(randint(0, 10) > 5):
        input = FT.vflip(input);
        target = FT.vflip(target);
    if(randint(0, 10) > 5):
        input = FT.hflip(input);
        target = FT.hflip(target);
    angle = randint(-30, 30);
    input = FT.rotate(input, angle, interpolation = torchvision.transforms.InterpolationMode.BILINEAR);
    target = FT.rotate(target, angle, interpolation = torchvision.transforms.InterpolationMode.BILINEAR);
    return (input, target);

class SWIRDataset(Dataset):

    def __init__(self, root_dir: str, transform) -> None:
        self.root_dir = root_dir;
        self.transform = transform;
        self.img_64 = [];
        self.img_256 = [];
        for idx in range(715):
            self.img_64.append(torch.cat([FT.to_tensor(Image.open(os.path.join(self.root_dir, "64", "800", f"{(idx + 1):04}.jpg")).convert('L')),
                                          FT.to_tensor(Image.open(os.path.join(self.root_dir, "64", "1050", f"{(idx + 1):04}.jpg")).convert('L')),
                                          FT.to_tensor(Image.open(os.path.join(self.root_dir, "64", "1550", f"{(idx + 1):04}.jpg")).convert('L'))], dim = 0));
            self.img_256.append(torch.cat([FT.to_tensor(Image.open(os.path.join(self.root_dir, "256", "800", f"{(idx + 1):04}.jpg")).convert('L')),
                                           FT.to_tensor(Image.open(os.path.join(self.root_dir, "256", "1050", f"{(idx + 1):04}.jpg")).convert('L')),
                                           FT.to_tensor(Image.open(os.path.join(self.root_dir, "256", "1550", f"{(idx + 1):04}.jpg")).convert('L'))], dim = 0));
    
    def __len__(self) -> int:
        return 715;

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        img_64, img_256 = self.transform(self.img_64[idx], self.img_256[idx]);
        return img_64, img_256;

In [5]:
class FFTLoss(nn.Module):

    def __init__(self, loss_weight: float = 1.0, reduction: str = 'mean') -> None:
        super().__init__();
        self.loss_weight = loss_weight;
        self.criterion = torch.nn.L1Loss(reduction = reduction);

    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        pred_fft = torch.fft.rfft2(pred);
        target_fft = torch.fft.rfft2(target);
        pred_fft = torch.stack([pred_fft.real, pred_fft.imag], dim = -1);
        target_fft = torch.stack([target_fft.real, target_fft.imag], dim = -1);
        return self.loss_weight * (self.criterion(pred_fft, target_fft));

In [6]:
def main():
    dataset = SWIRDataset('SWIR', transform);
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True);
    net = SRViT().train().to(device);
    opt_net = optim.Adam(net.parameters(), lr = learning_rate);
    fft = FFTLoss(0.1);
    avg_pix_loss = 0.0;
    avg_fft_loss = 0.0;
    it = 0;
    for i in tqdm(range(epochs)):
        for lr, hr in iter(dataloader):
            lr = lr.to(device);
            hr = hr.to(device);

            sr = net(lr);
            
            pix_loss = F.l1_loss(sr, hr);
            fft_loss = fft(sr, hr);
            avg_pix_loss += pix_loss.item();
            avg_fft_loss += fft_loss.item();
            total_loss = pix_loss + fft_loss;
            opt_net.zero_grad();
            total_loss.backward();
            opt_net.step();
            it += 1;
        if(((i + 1) % 100) == 0):
            print("epoch: {:d}, pix: {:f}, fft: {:f}".format(i + 1, avg_pix_loss / it, avg_fft_loss / it));
            torch.save(net.state_dict(), 'srvit_{:d}.pth'.format(i + 1));
            avg_pix_loss = 0.0;
            avg_fft_loss = 0.0;
            it = 0;
        if(((i + 1) % 1500) == 0):
            for param_group in opt_net.param_groups:
                param_group['lr'] /= 2.0;

main();

  1%|▏         | 100/8000 [06:17<8:15:50,  3.77s/it]

epoch: 100, pix: 0.091184, fft: 0.696591


  2%|▎         | 200/8000 [12:32<8:07:54,  3.75s/it]

epoch: 200, pix: 0.075961, fft: 0.646417


  4%|▍         | 300/8000 [18:48<8:02:36,  3.76s/it]

...

epoch: 7800, pix: 0.056473, fft: 0.590530


 99%|█████████▉| 7900/8000 [8:14:16<06:16,  3.76s/it]

epoch: 7900, pix: 0.056485, fft: 0.590250


100%|██████████| 8000/8000 [8:20:31<00:00,  3.75s/it]

epoch: 8000, pix: 0.056320, fft: 0.588702
